In [18]:
from datasets import load_dataset
from utils import mp3_compress, opus_compress, encodec_compress
from transformers import pipeline
from evaluate import evaluator
import encodec

In [4]:
device = "cuda"
encodec_model_48_6 = encodec.EncodecModel.encodec_model_48khz()
encodec_model_48_6.set_target_bandwidth(6)
encodec_model_48_6.to(device)
def encodec_48_6_compress(sample):
    return encodec_compress(sample, encodec_model_48_6, device)

In [15]:
audio_compression_methods = [
    mp3_compress,
    opus_compress,
    encodec_48_6_compress
]
models = [
    "openai/whisper-small"
]

In [6]:
common_voice = [load_dataset("mozilla-foundation/common_voice_11_0",
                            "en",
                            split="validation[:300]"
                           ).with_format("torch")]

In [7]:
for method in audio_compression_methods:
    common_voice.append(common_voice[0].map(method))

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [16]:
pipe = []
for model in models:
    pipe.append(
        pipeline(
            task="automatic-speech-recognition",
            model=model,
            device="cuda:0"
        )
    )

In [19]:
task_evaluator = evaluator("automatic-speech-recognition")
task_evaluator.PIPELINE_KWARGS.pop('truncation', None)

True

In [22]:
eval_results = []
for i_method,method in enumerate(common_voice):
    eval_results.append([])
    for i_model,model in enumerate(pipe):
        eval_results[i_method].append(
            task_evaluator.compute(
                model_or_pipeline=model,
                data=method.with_format("numpy"),
                input_column="audio",
                label_column="sentence",
                metric="wer",
            )
        )

In [24]:
[[rij['wer'] for rij in r] for r in eval_results]

[[0.17097902097902098],
 [0.2409090909090909],
 [0.2958041958041958],
 [0.26153846153846155]]